In [1]:
%load_ext autoreload
%autoreload 2
from fun import *

In [2]:
# read db using the gffutils library
db_filename = '/home/msarrias/dbs/homo_1.db'
db = gffutils.FeatureDB(db_filename, keep_order=True)
annot_dict = get_annotations_dict(db)

In [ ]:
# parent_dic, parent_child_dic = create_parent_child_dic(annot_dict)
# dump_parent_child_dict(parent_dic, parent_child_dic,'../files/parent_child_dic_human_01.pkl')


# genes parent - child annotations
parent_dic_gene, parent_child_dic_gene_mRNA = create_parent_child_dic(annot_dict,parent_type = ['gene'],
                                                       child_type = ['mRNA'])

# mRNA parent - child annotations
with open('../files/parent_child_dic_human_01.pkl', 'rb') as handle:
    merged_dict = pickle.load(handle)
    
mRNA_parent_dic, parents_exon_coverage = get_exon_overlaps(merged_dict['parent_child_dic'], 
                                                           merged_dict['parent_dic'])
# get gene - exons relationship
gene_hierarchy_dict = {}
for ID, value in parent_child_dic_gene_mRNA.items():
    dict_temp = {}
    for transcript, attrib in value['mRNA'].items():
        dict_temp[transcript] = parents_exon_coverage[transcript].copy()
    gene_hierarchy_dict[ID] = dict_temp


fasta_sequences = SeqIO.parse(open('/home/msarrias/data/Homo_sapiens.GRCh38.dna.chromosome.1.fa'),'fasta')
fasta_sequences = {fasta.id:str(fasta.seq) for fasta in fasta_sequences}

# get a sample of genes with 3 exons
sample_genes_3exons = {}
for gene_id, gene_attrib in gene_hierarchy_dict.items():
    # to avoid choosing between sources
    if len(gene_attrib) == 1:
        for key, value in gene_attrib.items():
            if len(value['exons']) == 3:
                temp_gene_attrib = gene_attrib.copy()
                for value in temp_gene_attrib.values():
                    start, end = value['parent']['coord'] 
                    value['parent']['seq'] = fasta_sequences['1'][start-1:end-1]
                    for value_exon in value['exons'].values():
                        start_e, end_e = value_exon['coord']
                        value_exon['seq'] = fasta_sequences['1'][start_e-1:end_e-1]
                sample_genes_3exons[gene_id] = temp_gene_attrib

In [ ]:
# example:

sample_genes_3exons.keys()

In [ ]:
sample_genes_3exons['gene:ENSG00000197921']